## CALCULO DE FUERZAS DE VIENTO SOBRE LA TORRE

### Datos de inicio

In [1]:
wind_speed_mph = 100   # Velocidad básica del viento en mph
V = wind_speed_mph * 0.44704       # Velocidad básica del viento en m/s
Kzt = 1.0              # Factor topográfico
Kd = 0.85              # Factor de direccionalidad
I = 1.0               # Factor de importancia
Gh = 0.85               # Factor de ráfaga 0.85 para estructuras de menos de 137 metros de altura.
Cf = 3.4               # Coeficiente de fuerza

In [2]:
# factores para el calculo de Kz, zg en metros
zg = 274
alfa = 9.50
kzmin = 0.85

### Geometria de las secciones de la torre

In [3]:
import pandas as pd
secciones = pd.ExcelFile(r"C:\Users\ahcas\Documents\Notebook\TIA-222-H\TORRE_45_AS.xlsx")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ahcas\\Documents\\Notebook\\TIA-222-H\\TORRE_45_AS.xlsx'

In [ ]:
df_secciones = secciones.parse("secciones")
df_secciones.style.hide(axis="index")

### Calculo de la presion equivalente

In [ ]:
# Funcion para calcular Kz
zg = float(zg)
alfa = float(alfa)
def calcular_kz(z,zg,alfa,kzmin):
    kzaux = 2.01 * ( z / zg )**(2/alfa)
    if kzaux > 2.01:
        kz = 2.01
    elif kzaux < kzmin:
        kz = kzmin
    else:
        kz = kzaux
    return kz

In [ ]:
# Función para calcular qz m/s
def calcular_qz_mps(Kz, Kzt, Kd, V, I):
    return 0.613 * Kz * Kzt * Kd * (V**2) * I

In [ ]:
pres = {}
pres['Seccion'] = df_secciones['Seccion']
pres['Z (m)'] = df_secciones['Alt. Med. (m)']

In [ ]:
df_pres = pd.DataFrame(pres)
# Aplicar la función a la columna "Alt. Inf (m)"
df_pres["Kz"] = df_secciones["Alt. Med. (m)"].apply(lambda z: calcular_kz(z, zg, alfa, kzmin))

In [ ]:
df_pres["qz (N/m2)"] = df_pres["Kz"].apply(lambda kz: calcular_qz_mps(kz, Kzt, Kd, V, I))

In [ ]:
df_pres.style.hide(axis="index")

### Factores de direccion del viento

In [ ]:
# Componentes Planos
df_00 = 1
df_60 = 0.80
df_90 = 0.85

# Componentes Circulares
dr_00 = 1
dr_60 = 1
dr_90 = 1

### Calculo de areas de los elementos

In [ ]:
areacalc = {}
areacalc['Seccion'] = df_secciones['Seccion']
areacalc['Areas Planas (m2)'] = df_secciones['Long. Diag. (m)']*df_secciones['Ancho Diag. (m)'] + df_secciones['Long. Hor. (m)']*df_secciones['Ancho Hor. (m)']
areacalc['Areas Circulares (m2)'] = df_secciones['Long. Pierna (m)']*df_secciones['Ancho Pierna (m)']
areacalc['Area Bruta (m2)'] = df_secciones['Altura Secc. (m)']* ( df_secciones['Ancho Inf. (m)'] + df_secciones['Ancho Sup. (m)'] )*0.5
areacalc['Relacion de Solidez'] = (areacalc['Areas Planas (m2)'] + areacalc['Areas Circulares (m2)']) / areacalc['Area Bruta (m2)']
areacalc['Coef. de Fuerza'] = 3.4 * areacalc['Relacion de Solidez']**2 - 4.7 * areacalc['Relacion de Solidez'] + 3.4
areacalc['Red. Factor'] = 0.57 - 0.14 * areacalc['Relacion de Solidez'] + 0.86 * areacalc['Relacion de Solidez']**2 - 0.24 * areacalc['Relacion de Solidez']**3
areacalc['Ar*Rr'] = areacalc['Areas Circulares (m2)'] * areacalc['Red. Factor']

In [ ]:
df_areacalc = pd.DataFrame(areacalc)

In [ ]:
df_areacalc.style.hide(axis="index")

### Areas proyectadas efectivas

In [ ]:
area_proy = {}
area_proy['Seccion'] = df_secciones['Seccion']

In [ ]:
area_proy['(EPA)s_00 (m2)'] = areacalc['Coef. de Fuerza'] * ( df_00 * areacalc['Areas Planas (m2)'] + dr_00 * areacalc['Ar*Rr'])
area_proy['(EPA)s_60 (m2)'] = areacalc['Coef. de Fuerza'] * ( df_60 * areacalc['Areas Planas (m2)'] + dr_60 * areacalc['Ar*Rr'])
area_proy['(EPA)s_90 (m2)'] = areacalc['Coef. de Fuerza'] * ( df_90 * areacalc['Areas Planas (m2)'] + dr_90 * areacalc['Ar*Rr'])

In [ ]:
df_area_proy = pd.DataFrame(area_proy)

In [ ]:
df_area_proy.style.hide(axis="index")

### Calculo de las fuerzas de viento

In [ ]:
fuerza_torre = {}
fuerza_torre['Seccion'] = df_secciones['Seccion']

In [ ]:
fuerza_torre['FST_00 (N)'] = df_pres['qz (N/m2)'] * Gh * df_area_proy['(EPA)s_00 (m2)']
fuerza_torre['FST_60 (N)'] = df_pres['qz (N/m2)'] * Gh * df_area_proy['(EPA)s_60 (m2)']
fuerza_torre['FST_90 (N)'] = df_pres['qz (N/m2)'] * Gh * df_area_proy['(EPA)s_90 (m2)']

In [ ]:
df_fuerza_torre = pd.DataFrame(fuerza_torre)

In [ ]:
df_fuerza_torre.style.hide(axis="index")